---
# 환경 설정
---

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install mxnet
!pip install gluonnlp tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-3.0.2-py3-none-any.whl (769 kB)
  Using cached tokenizers-0.8.1rc1-cp37-cp37m-manylinux1_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.10.3
    Uninstalling tokenizers-0.10.3:
      Successfully uninstalled tokenizers-0.10.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.8.1
    Uninstalling transformers-4.8.1:
      Successfully uninstalled transformers-4.8.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# !git clone https://github.com/SKTBrain/KoBERT.git

In [4]:
cd /content/drive/MyDrive/aiffel/NLP/KoBERT

/content/drive/MyDrive/aiffel/NLP/KoBERT


In [5]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.8.1-py3-none-any.whl (2.5 MB)
  Using cached tokenizers-0.10.3-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (3.3 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.8.1rc1
    Uninstalling tokenizers-0.8.1rc1:
      Successfully uninstalled tokenizers-0.8.1rc1
  Attempting uninstall: transformers
    Found existing installation: transformers 3.0.2
    Uninstalling transformers-3.0.2:
      Successfully uninstalled transformers-3.0.2


In [6]:
# pip install torchmetrics

In [7]:
pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
import wandb

wandb.login()


wandb: Currently logged in as: glad415. Use `wandb login --relogin` to force relogin


True

In [9]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [10]:
import pandas as pd 
from sklearn.model_selection import train_test_split
import re
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

In [11]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [12]:
# GPU 사용
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [13]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /content/drive/MyDrive/aiffel/NLP/KoBERT/.cache/kobert_v1.zip
using cached model. /content/drive/MyDrive/aiffel/NLP/KoBERT/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


---
# 학습용 데이터셋 전처리
---

In [14]:
train1 = pd.read_csv('/content/drive/MyDrive/aiffel/AIFFELTON/for_model_labeling_folder/model_labeling_0826_young_train.csv')
train2 = pd.read_csv('/content/drive/MyDrive/aiffel/AIFFELTON/for_model_labeling_folder/model_labeling_0826_keon_train.csv')
train3 = pd.read_csv('/content/drive/MyDrive/aiffel/AIFFELTON/for_model_labeling_folder/model_labeling_0829_jang_train.csv')
train4 = pd.read_csv('/content/drive/MyDrive/aiffel/AIFFELTON/for_model_labeling_folder/model_labeling_0829_young_train.csv')


In [15]:
del train1['Column1']
del train2['Unnamed: 0']
del train3['Unnamed: 0']
del train4['Column1']

In [16]:
data = pd.concat([train1,train2,train3],axis = 0, ignore_index = True)
data

,review,label
0,배달의 맛집을 경험했습니다 돈까스 싫어 하는데 딸이 시켜서 맛본다고 먹었죠 튀김옷도...,1
1,항상 맛있게 잘먹었습니ㅎㅎ,1
2,맛있어요 특히 소스가 맛있네요 너무 달고 신 소스가 아니고 은근히 댕기는 맛에 소스...,1
3,너무맛있어요 서비스 과자도 넘 감사합니당,1
4,따뜻하게 오고 음식도 너무너무 맛있어서 재주문하게 되는 곳이에요,1
...,...,...
22070,복권 새우링이 안왔다 당연히 있을줄알고 쿠폰도 줬는데,3
22071,가오점은 첨시켯는데 맛은있지만 도우 많이두꺼워서 쫌 두판시켜 다먹엇네요 양파,"1,2"
22072,피클 상했음 피클상함 냄새 쉼 맛도 쉼 오븐스파게티 별로 ㅠ,1
22073,한시간만에 온 피자가 다 식어서 치즈가 맛없게 굳어있네요 다시 오븐렌지에 4분이상 ...,3


In [17]:
data[['taste','quantity','delivery','else']] = np.NaN

In [18]:
data['label'].unique()

array(['1', '2', '3', '4', '1,2', '1,2,3', '1,2,3,4', '1,2,4', '1,3',
       '1,3,4', '1,4', '2,3', '2,3,4', '2,4', '3,4'], dtype=object)

In [19]:
data['label'] = data['label'].apply(lambda x : x.split(','))

In [20]:
data = data.reset_index(drop = True)

In [21]:
for i in range(len(data)):
  if '1' in data['label'][i]:
    data['taste'][i] = 1
  else :
    data['taste'][i] = 0

  if '2' in data['label'][i]:
    data['quantity'][i] = 1
  else :
    data['quantity'][i] = 0

  if '3' in data['label'][i]:
    data['delivery'][i] = 1
  else :
    data['delivery'][i] = 0

  if '4' in data['label'][i]:
    data['else'][i] = 1
  else :
    data['else'][i] = 0        

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

In [22]:
del data['label']
del data['else']

In [23]:
data[['taste','quantity','delivery']] = data[['taste','quantity','delivery']].astype(int)

In [24]:
data = pd.concat([data,train4],axis = 0, ignore_index = True)
data

,review,taste,quantity,delivery
0,배달의 맛집을 경험했습니다 돈까스 싫어 하는데 딸이 시켜서 맛본다고 먹었죠 튀김옷도...,1,0,0
1,항상 맛있게 잘먹었습니ㅎㅎ,1,0,0
2,맛있어요 특히 소스가 맛있네요 너무 달고 신 소스가 아니고 은근히 댕기는 맛에 소스...,1,0,0
3,너무맛있어요 서비스 과자도 넘 감사합니당,1,0,0
4,따뜻하게 오고 음식도 너무너무 맛있어서 재주문하게 되는 곳이에요,1,0,0
...,...,...,...,...
26555,할인까지 받아서 정말 맛나게 머것어요,1,0,0
26556,오랜만에 자메이카통다리 주문 밥이랑 먹기 좋았어요,1,0,0
26557,황금올리브 닭다리는 언제나 옳아요 맛있어요,1,0,0
26558,맛도좋구배달도빠르네요,1,0,1


In [25]:
data = data.drop_duplicates(subset = 'review', keep = 'first',ignore_index=True)

In [26]:
# data.to_csv('/content/drive/MyDrive/aiffel/AIFFELTON/for_model_labeling_folder/total_train_0829.csv')

In [27]:
data_list = [] # 최종 데이터셋 리스트 
for q, taste_label, quantity_label, delivery_label in zip(data['review'], data['taste'], data['quantity'], data['delivery']) :
  dataset = []  # 라벨과 내용이 하나의 리스트에 넣기 위해서 빈 리스트 생성
  dataset.append(q)
  dataset.append(str(taste_label))
  dataset.append(str(quantity_label))
  dataset.append(str(delivery_label))

  data_list.append(dataset)

In [28]:
data_list[:3]

[['배달의 맛집을 경험했습니다 돈까스 싫어 하는데 딸이 시켜서 맛본다고 먹었죠 튀김옷도 얇고 고기도 너무 부드러워서 놀랐네요 진짜 강추예요 앞으로 여기 돈까스만 먹을듯하네요',
  '1',
  '0',
  '0'],
 ['항상 맛있게 잘먹었습니ㅎㅎ', '1', '0', '0'],
 ['맛있어요 특히 소스가 맛있네요 너무 달고 신 소스가 아니고 은근히 댕기는 맛에 소스를 막 부어서 먹었답니다 음식에 사장님의 정성이 담뿍 들어있더라구요',
  '1',
  '0',
  '0']]

In [29]:
import numpy as np 

total_data_text = list(data['review'])
# 텍스트데이터 문장길이의 리스트를 생성한 후
num_tokens = [len(tokens) for tokens in total_data_text]
num_tokens = np.array(num_tokens)
# 문장길이의 평균값, 최대값, 표준편차를 계산해 본다. 
print('문장길이 평균 : ', np.mean(num_tokens))
print('문장길이 최대 : ', np.max(num_tokens))
print('문장길이 표준편차 : ', np.std(num_tokens))

# 예를들어, 최대 길이를 (평균 + 2*표준편차)로 한다면,  
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
maxlen = int(max_tokens)
print('pad_sequences maxlen : ', maxlen)
print('전체 문장의 {}%가 maxlen 설정값 이내에 포함됩니다. '.format(np.sum(num_tokens < max_tokens) / len(num_tokens)))

문장길이 평균 :  28.316577419111546
문장길이 최대 :  263
문장길이 표준편차 :  19.889821463104475
pad_sequences maxlen :  68
전체 문장의 0.9420770797194359%가 maxlen 설정값 이내에 포함됩니다. 


In [30]:
train_data, test_data = train_test_split(data_list, test_size = 0.2, random_state = 53)

---
# 모델
---

In [31]:
class BERTDataset(Dataset): 
  def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
    transform = nlp.data.BERTSentenceTransform(
        bert_tokenizer, max_seq_length = max_len, pad = pad, pair = pair)
    
    self.sentences = [transform([i[sent_idx]]) for i in dataset]
    self.taste = [np.int64(i[label_idx]) for i in dataset]
    self.quantity = [np.int64(i[label_idx+1]) for i in dataset]
    self.delivery = [np.int64(i[label_idx+2]) for i in dataset]

  def __getitem__(self,i):
    return (self.sentences[i] + (self.taste[i],self.quantity[i],self.delivery[i] ))

  def __len__(self):
    return (len(self.taste))

In [32]:
# 하이퍼파라미터 세팅
max_len = 68
batch_size = 32
warmup_ratio = 0.1
num_epochs = 2
max_grad_norm = 1
log_interval = 400
learning_rate =  4e-5

In [33]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False ,num_best = -1 )

using cached model. /content/drive/MyDrive/aiffel/NLP/KoBERT/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [34]:
data_train = BERTDataset(train_data, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(test_data, 0, 1, tok, max_len, True, False)

In [35]:
train_dataloader = torch.utils.data.DataLoader(data_train, shuffle=True, batch_size=batch_size, num_workers=2, drop_last=True)
test_dataloader = torch.utils.data.DataLoader(data_test, shuffle=False, batch_size=batch_size, num_workers=2, drop_last=True)

In [36]:
wandb.init(project = 'kobert',
                 entity="glad415",
                 config = {
                     "hidden_size" : 768,
                     "max_len" : 68,
                     "batch_size" : 32,
                     "learning_rate" : 4e-5,
                     "epochs" : 2,
                     "optimizer" : 'adam',
                     "loss" : "BCEloss",
                 })

In [37]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 ################# 클래스 수 조정 ##################
                 num_classes=1,   
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.taste = nn.Linear(hidden_size , num_classes)
        self.quantity = nn.Linear(hidden_size , num_classes)
        self.delivery = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

        self.sigmoid = nn.Sigmoid()
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, out = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(out)

        taste_out = self.taste(out)
        quantity_out = self.quantity(out)
        delivery_out = self.delivery(out)   

        return self.sigmoid(taste_out), self.sigmoid(quantity_out), self.sigmoid(delivery_out)

In [38]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.BCELoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [39]:
for a,b,c,d,e,f in train_dataloader:
  print(a.shape)
  print(b.shape)
  print(c.shape)
  print(d.shape)
  print(e.shape)
  print(f.shape)
  break

torch.Size([32, 68])
torch.Size([32])
torch.Size([32, 68])
torch.Size([32])
torch.Size([32])
torch.Size([32])


In [40]:
for e in range(num_epochs):
    train_acc = 0.0
    total_f1 = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, taste_label, quantity_label, delivery_label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length

        taste_label = taste_label.float().to(device)
        taste_label = taste_label.unsqueeze(1)

        quantity_label = quantity_label.float().to(device)
        quantity_label = quantity_label.unsqueeze(1)

        delivery_label = delivery_label.float().to(device)
        delivery_label = delivery_label.unsqueeze(1)

        taste_out, quantity_out, delivery_out = model(token_ids, valid_length, segment_ids)
         
        # Loss
        ################################################################################################
        taste_loss = loss_fn(taste_out, taste_label)
        quantity_loss = loss_fn(quantity_out, quantity_label)
        delivery_loss = loss_fn(delivery_out, delivery_label)

        total_loss = (taste_loss + quantity_loss + delivery_loss)
        total_loss.backward() 
        ################################################################################################

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

        optimizer.step()
        scheduler.step()  # Update learning rate schedule

        # taste_accuracy += calc_accuracy(taste_out, taste_label)
        # quantity_accuracy += calc_accuracy(quantity_out, quantity_label)
        # delivery_accuracy += calc_accuracy(delivery_out, delivery_label)

        # Accuracy
        ################################################################################################
        taste_preds = torch.argmax(taste_out, dim=1).flatten()
        quantity_preds = torch.argmax(quantity_out, dim=1).flatten()
        delivery_preds = torch.argmax(delivery_out, dim=1).flatten()

        taste_accuracy = (taste_preds == taste_label).cpu().numpy().mean()
        quantity_accuracy = (quantity_preds == quantity_label).cpu().numpy().mean()
        delivery_accuracy = (delivery_preds == delivery_label).cpu().numpy().mean()

        train_acc += (taste_accuracy + quantity_accuracy + delivery_accuracy) / 3
        ################################################################################################
        
        # F1_Score
        ################################################################################################
        taste_f1 = f1_score(taste_preds.cpu().data, taste_label.cpu(), average = 'micro')
        quantity_f1 = f1_score(quantity_preds.cpu().data, quantity_label.cpu(), average = 'micro')
        delivery_f1 = f1_score(delivery_preds.cpu().data, delivery_label.cpu(), average = 'micro')
        
        total_f1 +=  (taste_f1 + quantity_f1 + delivery_f1) / 3
        ################################################################################################

        wandb.log({'loss': total_loss.data.cpu().numpy(),
                    'taste_accuracy' :taste_accuracy, 'quantity_accuracy' : quantity_accuracy, 'delivery_accuracy' : delivery_accuracy,
                    'taste_f1' : taste_f1, 'quantity_f1' : quantity_f1, 'delivery_f1' : delivery_f1})
        if batch_id % log_interval == 0:

            print(f"epoch {e+1} batch id {batch_id+1} loss {total_loss.data.cpu().numpy():.3f} train acc {train_acc / (batch_id+1):.3f}")
            print(f'taste_accuracy {taste_accuracy:.3f}  | quantity_accuracy {quantity_accuracy:.3f}  | delivery_accuracy {delivery_accuracy:.3f}')
            print(f'f1_score:{total_f1 / (batch_id+1):.3f}','\n')
            print('#'*80)

    print(f"epoch {e+1} train acc {train_acc / (batch_id+1):.3f}, f1_score {total_f1 / (batch_id+1):.3f}")
    wandb.log({'train_acc' : train_acc / (batch_id+1), 'total_f1' : total_f1 / (batch_id+1)})
    # train_history.append(train_acc / (batch_id+1))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/662 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.056 train acc 0.615
taste_accuracy 0.438  | quantity_accuracy 0.812  | delivery_accuracy 0.594
f1_score:0.615 

################################################################################
epoch 1 batch id 401 loss 0.963 train acc 0.596
taste_accuracy 0.438  | quantity_accuracy 0.875  | delivery_accuracy 0.562
f1_score:0.596 

################################################################################
epoch 1 train acc 0.597, f1_score 0.597


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/662 [00:00<?, ?it/s]

epoch 2 batch id 1 loss 0.619 train acc 0.615
taste_accuracy 0.438  | quantity_accuracy 0.844  | delivery_accuracy 0.562
f1_score:0.615 

################################################################################
epoch 2 batch id 401 loss 0.438 train acc 0.598
taste_accuracy 0.344  | quantity_accuracy 0.844  | delivery_accuracy 0.594
f1_score:0.598 

################################################################################
epoch 2 train acc 0.597, f1_score 0.597


In [41]:
wandb.finish()

delivery_accuracy,▃▇▆▅▅▂▆▂▅▄▂▆▆▄▄▁▄▅▃█▂▅▄▄▅▄▂▃▂▄▃▄▃▅▁▁▅▇▃▃
delivery_f1,▃▇▆▅▅▂▆▂▅▄▂▆▆▄▄▁▄▅▃█▂▅▄▄▅▄▂▃▂▄▃▄▃▅▁▁▅▇▃▃
loss,██▆▄▄▃▃▃▃▁▂▃▂▃▂▂▂▂▂▃▂▄▃▂▂▃▂▁▃▂▁▃▃▂▂▁▂▂▁▄
quantity_accuracy,▄▃▇▇▄▆▁▆▅▇█▂▅▅▅▆▄▆▆▂▇▅▄▇▆▂▄▅▃▆▅▅▄▆▇▇▄▄▅▇
quantity_f1,▄▃▇▇▄▆▁▆▅▇█▂▅▅▅▆▄▆▆▂▇▅▄▇▆▂▄▅▃▆▅▅▄▆▇▇▄▄▅▇
taste_accuracy,▅▃▄▅▂▅▃▅▃▃▆▇▅▆▆▆▃▄▃▅▅▆▅▃▁▅▃▃▅▅▅▆▅▃▃█▂▃▅▅
taste_f1,▅▃▄▅▂▅▃▅▃▃▆▇▅▆▆▆▃▄▃▅▅▆▅▃▁▅▃▃▅▅▅▆▅▃▃█▂▃▅▅
total_f1,▁█
train_acc,▁█
delivery_accuracy,0.625
delivery_f1,0.625


In [42]:
model.eval()
test_f1 = 0.0
test_acc = 0.0
for batch_id, (token_ids, valid_length, segment_ids, taste_label, quantity_label, delivery_label) in enumerate(tqdm_notebook(test_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length

    taste_label = taste_label.float().to(device)
    taste_label = taste_label.unsqueeze(1)

    quantity_label = quantity_label.float().to(device)
    quantity_label = quantity_label.unsqueeze(1)

    delivery_label = delivery_label.float().to(device)
    delivery_label = delivery_label.unsqueeze(1)

    taste_out, quantity_out, delivery_out = model(token_ids, valid_length, segment_ids)

    # Accuracy
    ################################################################################################    
    taste_preds_test = torch.argmax(taste_out, dim=1).flatten()
    quantity_preds_test = torch.argmax(quantity_out, dim=1).flatten()
    delivery_preds_test = torch.argmax(delivery_out, dim=1).flatten()

    taste_accuracy_test = (taste_preds_test == taste_label).cpu().numpy().mean()
    quantity_accuracy_test = (quantity_preds_test == quantity_label).cpu().numpy().mean()
    delivery_accuracy_test = (delivery_preds_test == delivery_label).cpu().numpy().mean()

    test_acc += (taste_accuracy_test + quantity_accuracy_test + delivery_accuracy_test) / 3
    ################################################################################################ 

    # F1_Score
    ################################################################################################
    taste_f1_test = f1_score(taste_preds.cpu().data, taste_label.cpu(), average = 'micro')
    quantity_f1_test = f1_score(quantity_preds.cpu().data, quantity_label.cpu(), average = 'micro')
    delivery_f1_test = f1_score(delivery_preds.cpu().data, delivery_label.cpu(), average = 'micro')
    
    test_f1 +=  ((taste_f1_test + quantity_f1_test + delivery_f1_test) / 3)
    ################################################################################################

print(f"train acc {test_acc / (batch_id+1):.3f}, f1_score {test_f1/ (batch_id+1):.3f}")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


  0%|          | 0/165 [00:00<?, ?it/s]

train acc 0.597, f1_score 0.597


In [121]:
# torch.save(model, '/content/drive/MyDrive/aiffel/AIFFELTON/model_save/koBERT_Multi_label_0829.pt')

In [ ]:
# model = torch.load('/content/drive/MyDrive/aiffel/AIFFELTON/model_save/koBERT_Multi_label_0825_first.pt')

In [43]:
def predict(predict_sentence):

    data = [predict_sentence, '0', '0','0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, taste_label, quantity_label, delivery_label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length

        taste_label = taste_label.long().to(device)
        quantity_label = quantity_label.long().to(device)
        delivery_label = delivery_label.long().to(device)

        taste_out, quantity_out, delivery_out = model(token_ids, valid_length, segment_ids)

        for taste, quantity, delivery in zip(taste_out, quantity_out, delivery_out):

            logits_taste = taste
            logits_quantity = quantity
            logits_delivery = delivery

            logits_taste = logits_taste.detach().cpu().numpy()
            logits_quantity = logits_quantity.detach().cpu().numpy()
            logits_delivery = logits_delivery.detach().cpu().numpy()

            print('맛에 대한 점수:',logits_taste)
            print('양 or 서비스에 대한 점수:',logits_quantity)
            print('배달에 대한 점수:',logits_delivery)

---
# 결과값 테스트
---

In [44]:
import warnings
warnings.filterwarnings('ignore')

In [88]:
predict('맛이 없다')

맛에 대한 점수: [0.9922125]
양 or 서비스에 대한 점수: [0.00682687]
배달에 대한 점수: [0.00716566]


In [89]:
predict('서비스가 좋아서 다시 가고 싶어요')

맛에 대한 점수: [0.05460523]
양 or 서비스에 대한 점수: [0.96762764]
배달에 대한 점수: [0.02718446]


In [45]:
predict('돈가스 맛집이라해서 가봤는데 양이 너무 적어요')

맛에 대한 점수: [0.22454166]
양 or 서비스에 대한 점수: [0.98661834]
배달에 대한 점수: [0.01153113]


In [91]:
predict('음식은 겉바속촉 매장은 깔끔')

맛에 대한 점수: [0.08137822]
양 or 서비스에 대한 점수: [0.00771843]
배달에 대한 점수: [0.61706215]


In [50]:
predict('가까워서 빠른 배달이 좋아요 상하이 맛있구요')

맛에 대한 점수: [0.9146554]
양 or 서비스에 대한 점수: [0.00871582]
배달에 대한 점수: [0.96769977]


In [47]:
predict('콜라가 거의 다 넘쳤어요 조금 흐른 정도가 아니라 쏟아진 수준이라 감튀도 축축해지고 ㅜㅜ')

맛에 대한 점수: [0.03825044]
양 or 서비스에 대한 점수: [0.11719188]
배달에 대한 점수: [0.976915]


In [51]:
predict('맛 양 배달 모두 만족합니다')

맛에 대한 점수: [0.8850258]
양 or 서비스에 대한 점수: [0.9668646]
배달에 대한 점수: [0.92154515]


In [46]:
predict('기사님 너무 친절하시고 감자도 꾹꾹 담아서 가득 주셔서 배부르게 먹었어요 콜라도 뚜껑 이중으로 해주셔서 하나도 안 새고 왔습니당')

맛에 대한 점수: [0.05177966]
양 or 서비스에 대한 점수: [0.8399854]
배달에 대한 점수: [0.8269914]


In [48]:
predict('생각보다 빨리 와서 좋았어요 ㅎㅎ')

맛에 대한 점수: [0.03096332]
양 or 서비스에 대한 점수: [0.00471298]
배달에 대한 점수: [0.990724]


In [97]:
predict('다 좋은데 피자가 찌그러져서 왔네요')

맛에 대한 점수: [0.04959869]
양 or 서비스에 대한 점수: [0.00883994]
배달에 대한 점수: [0.9674883]


In [52]:
predict('사업 번창하세요')

맛에 대한 점수: [0.17382532]
양 or 서비스에 대한 점수: [0.04415375]
배달에 대한 점수: [0.02554429]


In [49]:
predict('다른집 들렸다 온것 같아요')

맛에 대한 점수: [0.04421622]
양 or 서비스에 대한 점수: [0.01142957]
배달에 대한 점수: [0.7966194]


----
# 비라벨링 -> 모델로 라벨링 
---

In [126]:
labeling = pd.read_csv('/content/drive/MyDrive/aiffel/AIFFELTON/for_model_labeling_folder/model_labeling_0829_jang_working.csv')
labeling.columns = ['1','review','label']
labeling

,1,review,label
0,6620,맛있었어요 돈가스 맛있는 맛,NaN
1,6621,배달1시간10분걸렸네요 점심시간도아닌데 만두랑돈가스는 차갑구요 매장에갓을때 맛있어서...,NaN
2,6622,감사합니다 잘 먹을께요,NaN
3,6623,두번시켜서 먹을만한곳은 아니에요 양도적고 맛도 그닥입니다,NaN
4,6624,면이 좀 붙어있었지만ㅠ 맛은 너무 맛있었어요,NaN
...,...,...,...
39322,46144,잘 먹었습니다 서비스로 날치알밥도 맛있게 잘 먹었어요,NaN
39323,46145,항상 여기서 시켜먹어요 다 좋아요,NaN
39324,46146,가성비 최고 맛도 웬만한가게보다 훨씬나아요 앞으로 자주먹을듯,NaN
39325,46147,배달빠르고 맛보기 무침족발 새콤달콤하니 맛있어요,NaN


In [127]:
del labeling['1']

In [136]:
labeling

,review,label
0,맛있었어요 돈가스 맛있는 맛,NaN
1,배달1시간10분걸렸네요 점심시간도아닌데 만두랑돈가스는 차갑구요 매장에갓을때 맛있어서...,NaN
2,감사합니다 잘 먹을께요,NaN
3,두번시켜서 먹을만한곳은 아니에요 양도적고 맛도 그닥입니다,NaN
4,면이 좀 붙어있었지만ㅠ 맛은 너무 맛있었어요,NaN
...,...,...
39322,잘 먹었습니다 서비스로 날치알밥도 맛있게 잘 먹었어요,NaN
39323,항상 여기서 시켜먹어요 다 좋아요,NaN
39324,가성비 최고 맛도 웬만한가게보다 훨씬나아요 앞으로 자주먹을듯,NaN
39325,배달빠르고 맛보기 무침족발 새콤달콤하니 맛있어요,NaN


In [137]:
label_list = []
for i in range(len(labeling)):
  label_list.append(labeling['review'][i])

In [138]:
def predict(predict_sentence):

    data = [predict_sentence, '0', '0','0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, taste_label, quantity_label, delivery_label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length

        taste_label = taste_label.long().to(device)
        quantity_label = quantity_label.long().to(device)
        delivery_label = delivery_label.long().to(device)

        taste_out, quantity_out, delivery_out = model(token_ids, valid_length, segment_ids)

        for logits_taste, logits_quantity, logits_delivery in zip(taste_out, quantity_out, delivery_out):

            logits_taste = logits_taste.detach().cpu().numpy()
            logits_quantity = logits_quantity.detach().cpu().numpy()
            logits_delivery = logits_delivery.detach().cpu().numpy()

    labeling = []
    if logits_taste >= 0.7:
      labeling.append('1')
    else :
      labeling.append('0')

    if logits_quantity >= 0.7:
      labeling.append('1')
    else :
      labeling.append('0')

    if logits_delivery >= 0.7:
      labeling.append('1')
    else :
      labeling.append('0')
    
    return labeling

In [139]:
multi_label = []
for corpus in label_list:
  value = predict(corpus)
  multi_label.append(value)
  

Exception ignored in: <function _releaseLock at 0x7f2c59ec0830>
Traceback (most recent call last):
  File "/usr/lib/python3.7/logging/__init__.py", line 221, in _releaseLock
    def _releaseLock():
KeyboardInterrupt


KeyboardInterrupt: ignored

In [135]:
muiti_label

NameError: ignored

In [ ]:
a = pd.DataFrame(label_list)
a.columns = ['review']
a

,review
0,예정시간에 맞춰서 배달되었고 양도 많았어요 맛있게 잘먹었습니다
1,요기요에서는 처음 주문 해보네요 역시나 맛있습니다 ㅎㅎㅎ
2,배달은 예상시간보다 빨리와야 제맛 간장찜닭도 맛잇고 김치찜닭도 맛잇고 담엔 둘중에 뭐먹지
3,닭이 순살로 와서 먹기 편했어요 살짝 짜고 매웠는데 매식이 다 그렇죠 양 겁나 많아...
4,보통맛이지만 매운거 못먹는 사람들은 매울 수 있으니 순한맛 추천드려요
...,...
4501,할인까지 받아서 정말 맛나게 머것어요
4502,오랜만에 자메이카통다리 주문 밥이랑 먹기 좋았어요
4503,황금올리브 닭다리는 언제나 옳아요 맛있어요
4504,맛도좋구배달도빠르네요


In [ ]:
b = pd.DataFrame(multi_label)
b.columns = ['taste','quantity','delivery']
b

,taste,quantity,delivery
0,1,1,1
1,1,0,0
2,1,0,1
3,1,1,0
4,1,0,0
...,...,...,...
4501,1,0,0
4502,1,0,0
4503,1,0,0
4504,1,0,1


In [ ]:
b['review'] = a['review']

In [ ]:
b = b.reindex( columns = ['review','taste','quantity','delivery'])
b

,review,taste,quantity,delivery
0,예정시간에 맞춰서 배달되었고 양도 많았어요 맛있게 잘먹었습니다,1,1,1
1,요기요에서는 처음 주문 해보네요 역시나 맛있습니다 ㅎㅎㅎ,1,0,0
2,배달은 예상시간보다 빨리와야 제맛 간장찜닭도 맛잇고 김치찜닭도 맛잇고 담엔 둘중에 뭐먹지,1,0,1
3,닭이 순살로 와서 먹기 편했어요 살짝 짜고 매웠는데 매식이 다 그렇죠 양 겁나 많아...,1,1,0
4,보통맛이지만 매운거 못먹는 사람들은 매울 수 있으니 순한맛 추천드려요,1,0,0
...,...,...,...,...
4501,할인까지 받아서 정말 맛나게 머것어요,1,0,0
4502,오랜만에 자메이카통다리 주문 밥이랑 먹기 좋았어요,1,0,0
4503,황금올리브 닭다리는 언제나 옳아요 맛있어요,1,0,0
4504,맛도좋구배달도빠르네요,1,0,1


In [ ]:
b.to_csv('/content/drive/MyDrive/aiffel/AIFFELTON/for_model_labeling_folder/check_jang_0829.csv')